<a href="https://colab.research.google.com/github/niekh-13/geodata-etl-workshop/blob/main/Introductie_GeoPandas_Workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introductie Pandas en GeoPandas: eenvoudige ETL scripting

In deze workshop voor Kaartviewer inspiratie dagen 2024 leer je de basis van Pandas en GeoPandas kennen door gebruik te maken van open data van Nederlandse netbeheerders en CBS.

In [ ]:
%%capture

# Install necessary packages
!pip install pandas geopandas shapely OWSlib wget

In [ ]:
%%capture

# Importeren van packages
import requests
import zipfile
import os
import io
from owslib.wfs import WebFeatureService
import wget

## Stap 1: Data downloaden van gekozen netbeheerder met Python

### Kies één van de netbeheerders en download hun dataset.


#### Liander



In [ ]:
# Download Liander
url = "https://www.liander.nl/-/media/files/open-data/kleinverbruikdata/kleinverbruiksgegevens-2024.zip"
response = requests.get(url)
delimiter = ';'

# Get filenames and paths for Liander
filename = 'Liander_kleinverbruiksgegevens_20240101.csv'
zip_path = url.split("/")[-1]

# Write response content in to zipfile
with open(zip_path, "wb") as f:
  f.write(response.content)

# Extract the csv file from zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
  zip_ref.extractall(".")

print(f"{filename} is downloaded")

#### Enexis


In [ ]:
# Download Enexis
url = "https://enxp433-oda01.s3.eu-west-1.amazonaws.com/kv/Enexis_kleinverbruiksgegevens_01012024.csv"
response = requests.get(url)
delimiter = ';'

# Get filename for Enexis
filename = url.split("/")[-1]

# Write response content in to csv file
with open(filename, "wb") as f:
  f.write(response.content)

print(f"{filename} is downloaded")


#### Stedin

In [ ]:
# Deze code voert een specifieke taak uit
!wget https://www.stedin.net/-/media/project/online/files/zakelijk/open-data/stedin-kleinverbruikgegevens-2024.csv

delimiter = '\t'

filename = 'stedin-kleinverbruikgegevens-2024.csv'

#### Coteq

In [ ]:
# Download Coteq
url = "https://d3a07q56iliqjn.cloudfront.net/web-uploads/Documenten/Open-data/CoteqNetbeheer_kleinverbruik_01012024.csv"
filename = wget.download(url)
delimiter = ';'

print(f"{filename} is downloaded")

In [ ]:
!head -n -2 CoteqNetbeheer_kleinverbruik_01012024.csv > tmp.txt && iconv --from-code=ISO-8859-1 --to-code=UTF-8 tmp.txt > CoteqNetbeheer_kleinverbruik_01012024.csv


## Stap 2: Netbeheer data inlezen via Pandas

In [ ]:
# Deze code voert een specifieke taak uit
import pandas as pd

columns = [
    "NETBEHEERDER", "NETGEBIED", "STRAATNAAM", "POSTCODE_VAN", "POSTCODE_TOT",
    "WOONPLAATS", "LANDCODE", "PRODUCTSOORT", "VERBRUIKSSEGMENT", "AANSLUITINGEN_AANTAL",
    "LEVERINGSRICHTING_PERC", "FYSIEKE_STATUS_PERC", "SOORT_AANSLUITING_PERC",
    "SOORT_AANSLUITING", "SJV_GEMIDDELD", "SJV_LAAG_TARIEF_PERC", "SLIMME_METER_PERC"
]

# Inlezen van netbeheerder data met pandas
data = pd.read_csv(filename, sep=delimiter, dtype=str, names=columns, skiprows=1)

# Data van netbeheerder uniform maken voor pandas
data = data.map(lambda x: x.replace(',', '.') if isinstance(x, str) else x)

# Data van netbeheerder strippen van whitespace
data = data.map(lambda x: x.strip() if isinstance(x, str) else x)

#### Controleer data

In [ ]:
# Deze code voert een specifieke taak uit
print(data.info())
# print(data.describe())

## Stap 3: Netbeheer data manipuleren en voorbereiden voor analyse

In [ ]:
# Stap 1: Postcode5 afleiden
data['POSTCODE5'] = data['POSTCODE_TOT'].str.replace(' ', '').str[:5]


# Zorg dat AANSLUITINGEN_AANTAL en andere numerieke kolommen numeriek zijn
data['AANSLUITINGEN_AANTAL'] = pd.to_numeric(data['AANSLUITINGEN_AANTAL'], errors='coerce')
data['SJV_GEMIDDELD'] = pd.to_numeric(data['SJV_GEMIDDELD'], errors='coerce')
data['SJV_LAAG_TARIEF_PERC'] = pd.to_numeric(data['SJV_LAAG_TARIEF_PERC'], errors='coerce')
data['LEVERINGSRICHTING_PERC'] = pd.to_numeric(data['LEVERINGSRICHTING_PERC'], errors='coerce')
data['FYSIEKE_STATUS_PERC'] = pd.to_numeric(data['FYSIEKE_STATUS_PERC'], errors='coerce')
data['SOORT_AANSLUITING_PERC'] = pd.to_numeric(data['SOORT_AANSLUITING_PERC'], errors='coerce')
data['SLIMME_METER_PERC'] = pd.to_numeric(data['SLIMME_METER_PERC'], errors='coerce')

# # Stap 2: Functie om gewogen gemiddelde te berekenen
def weighted_average(df, col, weight_col):
    return df[col].sum() / df[weight_col].sum()

# # Stap 3: Groeperen op de gewenste kolommen en berekeningen uitvoeren
grouped_data = data.groupby(by=['POSTCODE5', 'PRODUCTSOORT', 'NETBEHEERDER', 'NETGEBIED', 'WOONPLAATS', 'LANDCODE', 'VERBRUIKSSEGMENT']
).apply(
    lambda x: pd.Series({
    'AANSLUITINGEN_TOTAAL': x['AANSLUITINGEN_AANTAL'].sum(),
    'SJV_GEMIDDELD_PC5': weighted_average(x, 'SJV_GEMIDDELD', 'AANSLUITINGEN_AANTAL'),
    'SJV_LAAG_TARIEF_PERC_PC5': weighted_average(x, 'SJV_LAAG_TARIEF_PERC', 'AANSLUITINGEN_AANTAL'),
    'LEVERINGSRICHTING_PERC_PC5': weighted_average(x, 'LEVERINGSRICHTING_PERC', 'AANSLUITINGEN_AANTAL'),
    'FYSIEKE_STATUS_PERC_PC5': weighted_average(x, 'FYSIEKE_STATUS_PERC', 'AANSLUITINGEN_AANTAL'),
    'SOORT_AANSLUITING_PERC_PC5': weighted_average(x, 'SOORT_AANSLUITING_PERC', 'AANSLUITINGEN_AANTAL'),
    'SLIMME_METER_PERC_PC5': weighted_average(x, 'SLIMME_METER_PERC', 'AANSLUITINGEN_AANTAL'),
})).reset_index()

In [ ]:
# Stap 4: Bekijk de gegroepeerde data
print(grouped_data.info())

## Stap 4: CBS Postcode Data downloaden

In [ ]:
# Make variables for download
pc5_url = "https://download.cbs.nl/postcode/2024-cbs_pc5_2023_v1.zip"
pc5_dirname = "CBS_Postcode" # Name of the directory

# Download CBS Postcode data
response = requests.get(pc5_url)

# Get filename for CBS Postcode
filename = pc5_url.split("/")[-1]

# Write response content in to zip file
with open(filename, "wb") as f:
    f.write(response.content)

# Extract the files from zip file
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall(f"./{pc5_dirname}")

print(f"{pc5_dirname} data is gedownload en uitgepakt")

## Stap 5: CBS Postcode data inlezen

In [ ]:
# Deze code importeerd gepandas
import geopandas as gpd

# Bestandspad naar het GeoPackage-bestand
cbs_postcode_file = "CBS_Postcode/cbs_pc5_2023_v1.gpkg"

# CBS Postcode data inlezen
cbs_postcode = gpd.read_file(cbs_postcode_file, layer='cbs_pc5_2023')

# Alleen de kolommen 'postcode' en 'geometry' selecteren
cbs_postcode = cbs_postcode[['postcode', 'geometry']]

#### Controleer data

In [ ]:
# print de dataframe uit om te controleren
print(cbs_postcode.head())

## Stap 6: CBS Postcode Data koppelen aan netbeheer data


In [ ]:
# Merging the datasets
merged_data = pd.merge(grouped_data, cbs_postcode, left_on="POSTCODE5", right_on="postcode", how="left")
merged_data = merged_data.drop(columns=["postcode"])
merged_data = gpd.GeoDataFrame(merged_data, geometry='geometry')

#### Controleer data

In [ ]:
# print(merged_data.head())
print(merged_data.info())

## Stap 7: CBS wijkbuurten kaart data downloaden voor gemeenten

In [ ]:
import geopandas as gpd
import requests
from owslib.wfs import WebFeatureService

# WFS URL
wfs_url = 'https://service.pdok.nl/cbs/wijkenbuurten/2023/wfs/v1_0'

# Stel de parameters voor het GET-verzoek
params = {
    'service': 'WFS',
    'version': '2.0.0',
    'request': 'GetFeature',
    'typeName': 'gemeenten',
    'outputFormat': 'json',
    'PropertyName': 'gemeentenaam,gemeentecode'
}

# Stel de headers voor het GET-verzoek
headers = {
    'Accept-Encoding': 'gzip'
}

# Maak een GET-verzoek met compressie
r = requests.get(wfs_url, params=params, headers=headers)

# Controleer of het verzoek succesvol was
if r.status_code == 200:
    print("Data succesvol opgehaald!")

    # Zet de JSON-data om naar een GeoDataFrame
    cbs_gemeente = gpd.read_file(io.BytesIO(r.content))

    # Filter de nodige kolommen
    # cbs_gemeente = cbs_gemeente[['gemeentecode', 'gemeentenaam', 'geometry']]
else:
    print(f"Fout bij het ophalen van de data: {r.status_code}")
    print(r.text)


#### Controleer data

In [ ]:
# print(cbs_gemeente.head())
print(cbs_gemeente.info())

## Stap 8: Gemeente koppelen aan postcode 5 cijfers

In [ ]:
# Deze code voert een specifieke taak uit
cbs_gemeente['geometry_right'] = cbs_gemeente.loc[:, 'geometry']

# Ruimtelijke join uitvoeren om de gemeentes te vinden die overlappen met postcodes
joined_data = gpd.sjoin(merged_data, cbs_gemeente, predicate='intersects')

# Bereken de overlappingsgebieden door een geometrische intersectie te maken tussen de postcodes en gemeentes
joined_data['intersection'] = joined_data.geometry_right.intersection(joined_data['geometry'])

# Bereken de oppervlakte van de intersectie
joined_data['intersection_area'] = joined_data['intersection'].area

# Reset de index zodat de later de juiste rijen geselecteerd kunnen worden
joined_data.reset_index(drop=True, inplace=True)

# Groepeer per postcode en kies de gemeente met het grootste overlappingsgebied
idx = joined_data.groupby(
    ['POSTCODE5', 'PRODUCTSOORT']
    )['intersection_area'].idxmax()

# Selecteer alleen de rijen met de grootste overlap per postcode
largest_overlap = joined_data.loc[idx]

# Rename geometry columns to have 'geometry' for the original geometry (geometry_left)
largest_overlap = largest_overlap.rename(columns={'geometry_left': 'geometry'})

# Behoud de gewenste kolommen
result = largest_overlap[['POSTCODE5', 'PRODUCTSOORT', 'NETBEHEERDER', 'NETGEBIED', 'WOONPLAATS',
                          'LANDCODE', 'VERBRUIKSSEGMENT', 'AANSLUITINGEN_TOTAAL', 'SJV_GEMIDDELD_PC5',
                          'SJV_LAAG_TARIEF_PERC_PC5', 'LEVERINGSRICHTING_PERC_PC5', 'FYSIEKE_STATUS_PERC_PC5',
                          'SOORT_AANSLUITING_PERC_PC5', 'SLIMME_METER_PERC_PC5', 'geometry',
                          'gemeentecode', 'gemeentenaam']]

#### Filter gemeente

In [ ]:
# Filter merged_data met de gemeente naar keuze
result = result[(result['gemeentecode'] == 'GM0228')].copy()

#### Controleren

In [ ]:
# Print the updated merged_data
print(result)


In [ ]:
# Print dubbelen rijen die eventeel voorkomen
duplicate_rows = result[result.duplicated(subset=['POSTCODE5', 'PRODUCTSOORT'], keep=False)]
print(duplicate_rows)

## Stap 9: Elektriciteit en gas data splitsen

In [ ]:
# Split into electricity and gas dataframes
electricity_data = result[result['PRODUCTSOORT'] == 'ELK']
gas_data = result[result['PRODUCTSOORT'] == 'GAS']

# Converteer naar GeoDataFrames
electricity_gdf = gpd.GeoDataFrame(electricity_data, geometry='geometry')
gas_gdf = gpd.GeoDataFrame(gas_data, geometry='geometry')


#### Controleer

In [ ]:
# Print nieuwe dataframes uit
print(electricity_gdf.info())
print(gas_gdf.info())

## Stap 10: Geodata output naar .gpkg bestand

In [ ]:
# Creeer een GeoPackage bestand
output_filename = 'netbeheerder_data.gpkg'

# Schrijf de twee GeoDataFrames naar het GeoPackage bestand
electricity_gdf.to_file(output_filename, layer='electricity', driver='GPKG')
gas_gdf.to_file(output_filename, layer='gas', driver='GPKG')

print(f"GeoPackage file '{output_filename}' created successfully.")